# Guess the Calories 🤤

**Name(s)**: Luran Zhang, Yunpeng Zhao

**Website Link**: https://elijahzyp.github.io/recipes_ratings_model/

## Code

In [1]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
pd.options.plotting.backend = 'plotly'

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Binarizer


### Framing the Problem

In [3]:
# load in the raw data for recipes and ratings (from project 3)
raw_recipes = pd.read_csv(os.path.join('data', 'RAW_recipes.csv'))
raw_interactions = pd.read_csv(os.path.join('data', 'RAW_interactions.csv'))
raw_recipes.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,1 brownies in the world best ever,333281,40,985201,2008-10-27,"['60-minutes-or-less', 'time-to-make', 'course...","[138.4, 10.0, 50.0, 3.0, 3.0, 19.0, 6.0]",10,['heat the oven to 350f and arrange the rack i...,"these are the most; chocolatey, moist, rich, d...","['bittersweet chocolate', 'unsalted butter', '...",9
1,1 in canada chocolate chip cookies,453467,45,1848091,2011-04-11,"['60-minutes-or-less', 'time-to-make', 'cuisin...","[595.1, 46.0, 211.0, 22.0, 13.0, 51.0, 26.0]",12,"['pre-heat oven the 350 degrees f', 'in a mixi...",this is the recipe that we use at my school ca...,"['white sugar', 'brown sugar', 'salt', 'margar...",11
2,412 broccoli casserole,306168,40,50969,2008-05-30,"['60-minutes-or-less', 'time-to-make', 'course...","[194.8, 20.0, 6.0, 32.0, 22.0, 36.0, 3.0]",6,"['preheat oven to 350 degrees', 'spray a 2 qua...",since there are already 411 recipes for brocco...,"['frozen broccoli cuts', 'cream of chicken sou...",9
3,millionaire pound cake,286009,120,461724,2008-02-12,"['time-to-make', 'course', 'cuisine', 'prepara...","[878.3, 63.0, 326.0, 13.0, 20.0, 123.0, 39.0]",7,"['freheat the oven to 300 degrees', 'grease a ...",why a millionaire pound cake? because it's su...,"['butter', 'sugar', 'eggs', 'all-purpose flour...",7
4,2000 meatloaf,475785,90,2202916,2012-03-06,"['time-to-make', 'course', 'main-ingredient', ...","[267.0, 30.0, 12.0, 12.0, 29.0, 48.0, 2.0]",17,"['pan fry bacon , and set aside on a paper tow...","ready, set, cook! special edition contest entr...","['meatloaf mixture', 'unsmoked bacon', 'goat c...",13


In [4]:
# merge recipes dataset and ratings dataset (from project 3)
raw_merged = raw_recipes.merge(raw_interactions, left_on='id', right_on='recipe_id', how='left')

In [5]:
# fill 0 with np.nan (from project 3)
merged_fillzero = raw_merged.copy()
merged_fillzero['rating'] = raw_merged['rating'].replace(0, np.nan)

In [6]:
# calculate the average rating per recipe (from project 3)
avg_rating = merged_fillzero.groupby("id")[["rating"]].mean()
avg_rating = avg_rating.rename(columns = {"rating":"avg_rating"})

In [7]:
# adding the avergae rating series back to the recipes dataset (from project 3)
recipes = raw_recipes.merge(avg_rating, on = "id")
recipes.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,avg_rating
0,1 brownies in the world best ever,333281,40,985201,2008-10-27,"['60-minutes-or-less', 'time-to-make', 'course...","[138.4, 10.0, 50.0, 3.0, 3.0, 19.0, 6.0]",10,['heat the oven to 350f and arrange the rack i...,"these are the most; chocolatey, moist, rich, d...","['bittersweet chocolate', 'unsalted butter', '...",9,4.0
1,1 in canada chocolate chip cookies,453467,45,1848091,2011-04-11,"['60-minutes-or-less', 'time-to-make', 'cuisin...","[595.1, 46.0, 211.0, 22.0, 13.0, 51.0, 26.0]",12,"['pre-heat oven the 350 degrees f', 'in a mixi...",this is the recipe that we use at my school ca...,"['white sugar', 'brown sugar', 'salt', 'margar...",11,5.0
2,412 broccoli casserole,306168,40,50969,2008-05-30,"['60-minutes-or-less', 'time-to-make', 'course...","[194.8, 20.0, 6.0, 32.0, 22.0, 36.0, 3.0]",6,"['preheat oven to 350 degrees', 'spray a 2 qua...",since there are already 411 recipes for brocco...,"['frozen broccoli cuts', 'cream of chicken sou...",9,5.0
3,millionaire pound cake,286009,120,461724,2008-02-12,"['time-to-make', 'course', 'cuisine', 'prepara...","[878.3, 63.0, 326.0, 13.0, 20.0, 123.0, 39.0]",7,"['freheat the oven to 300 degrees', 'grease a ...",why a millionaire pound cake? because it's su...,"['butter', 'sugar', 'eggs', 'all-purpose flour...",7,5.0
4,2000 meatloaf,475785,90,2202916,2012-03-06,"['time-to-make', 'course', 'main-ingredient', ...","[267.0, 30.0, 12.0, 12.0, 29.0, 48.0, 2.0]",17,"['pan fry bacon , and set aside on a paper tow...","ready, set, cook! special edition contest entr...","['meatloaf mixture', 'unsmoked bacon', 'goat c...",13,5.0


In [8]:
# first we expanded the nutrition columns into its corsponsing categories. (from project 3)
cleaned_recipes = recipes.copy()
cleaned_recipes[["calories (#)", 
          "total fat (PDV)", 
          "sugar (PDV)", 
          "sodium (PDV)", 
          "protein (PDV)", 
          "saturated fat (PDV)", 
          "carbohydrates (PDV)"]] = pd.DataFrame(recipes["nutrition"]\
                                                 .str.strip("[]").str.split(", ").to_list(), 
                         columns = ["calories (#)", "total fat (PDV)", "sugar (PDV)", 
                                    "sodium (PDV)", "protein (PDV)", 
                                    "saturated fat (PDV)", "carbohydrates (PDV)"]).astype(float)

cleaned_recipes = cleaned_recipes.drop(columns = "nutrition")

In [9]:
# add a boolean column that determines whether a recipe is healthy or not (from project 3)
avg = cleaned_recipes.loc[:, ["protein (PDV)", "carbohydrates (PDV)"]].mean()
healthy_rating = cleaned_recipes.loc[:, ["protein (PDV)", "carbohydrates (PDV)"]] <= avg
healthy_rating["protein (PDV)"] = healthy_rating["protein (PDV)"] == False
cleaned_recipes['healthy'] = healthy_rating.sum(axis=1) == 2

In [10]:
# Here we defined a function to help us determine if a person is happy after eating the 
# dish they cooked using each recipe. (from project 3)
def happiness_sort(row):
    if (row["avg_rating"] >= 1) & (row["avg_rating"] < 3):
        return "sad"
    if (row["avg_rating"] >= 3) & (row["avg_rating"] <= 5):
        return "happy"
    return np.nan

cleaned_recipes["happiness"] = cleaned_recipes.apply (lambda row: happiness_sort(row), axis=1)

In [11]:
# change the type of the submitted column to pandas datetime type. (from project 3)
cleaned_recipes["submitted"] = pd.to_datetime(cleaned_recipes["submitted"], format='%Y-%m-%d')
cleaned_recipes["published_year"] = cleaned_recipes["submitted"].dt.year

In [12]:
# set cooking time outliers and 0 to np.nan (from project 3)
cleaned_recipes["minutes"] = cleaned_recipes["minutes"].where(cleaned_recipes["minutes"] <= 1440)
cleaned_recipes["minutes"] = cleaned_recipes["minutes"].replace(0, np.nan)

In [13]:
# here we want to drop any row with np.nan because we can not use them to construct our model
cleaned_recipes = cleaned_recipes.dropna()
cleaned_recipes.head()

,name,id,minutes,contributor_id,submitted,tags,n_steps,steps,description,ingredients,...,calories (#),total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),healthy,happiness,published_year
0,1 brownies in the world best ever,333281,40.0,985201,2008-10-27,"['60-minutes-or-less', 'time-to-make', 'course...",10,['heat the oven to 350f and arrange the rack i...,"these are the most; chocolatey, moist, rich, d...","['bittersweet chocolate', 'unsalted butter', '...",...,138.4,10.0,50.0,3.0,3.0,19.0,6.0,False,happy,2008
1,1 in canada chocolate chip cookies,453467,45.0,1848091,2011-04-11,"['60-minutes-or-less', 'time-to-make', 'cuisin...",12,"['pre-heat oven the 350 degrees f', 'in a mixi...",this is the recipe that we use at my school ca...,"['white sugar', 'brown sugar', 'salt', 'margar...",...,595.1,46.0,211.0,22.0,13.0,51.0,26.0,False,happy,2011
2,412 broccoli casserole,306168,40.0,50969,2008-05-30,"['60-minutes-or-less', 'time-to-make', 'course...",6,"['preheat oven to 350 degrees', 'spray a 2 qua...",since there are already 411 recipes for brocco...,"['frozen broccoli cuts', 'cream of chicken sou...",...,194.8,20.0,6.0,32.0,22.0,36.0,3.0,False,happy,2008
3,millionaire pound cake,286009,120.0,461724,2008-02-12,"['time-to-make', 'course', 'cuisine', 'prepara...",7,"['freheat the oven to 300 degrees', 'grease a ...",why a millionaire pound cake? because it's su...,"['butter', 'sugar', 'eggs', 'all-purpose flour...",...,878.3,63.0,326.0,13.0,20.0,123.0,39.0,False,happy,2008
4,2000 meatloaf,475785,90.0,2202916,2012-03-06,"['time-to-make', 'course', 'main-ingredient', ...",17,"['pan fry bacon , and set aside on a paper tow...","ready, set, cook! special edition contest entr...","['meatloaf mixture', 'unsmoked bacon', 'goat c...",...,267.0,30.0,12.0,12.0,29.0,48.0,2.0,False,happy,2012


##### Prediction Problem
For our project, we hope to predict the **calories** for each recipe using feature engineering and modeling

### Baseline Model

In [14]:
# in order to test our model on unseen data, we will split our data into training and test data
X_train, X_test, y_train, y_test = train_test_split(cleaned_recipes, 
                                                    cleaned_recipes['calories (#)']
                                                   )

In [15]:
# Our base model includes two features, the year the recipe was published and the amount of fat 
# in the recipe.
preproc = ColumnTransformer(
        transformers = [
            ('year', OneHotEncoder(drop = "first"), ['published_year'])
            ],
        remainder='passthrough'
    )
pl = Pipeline([
        ("preprocessor", preproc),
        ('decision-tree', DecisionTreeRegressor(max_depth = 5))
    ])

pl.fit(X_train[["published_year", 'total fat (PDV)']], y_train)
pl.score(X_train[["published_year", 'total fat (PDV)']], y_train)

0.8119909202924103

In [16]:
#run base model on test data
pl.score(X_test[["published_year", 'total fat (PDV)']], y_test)

0.7252110171807438

### Final Model

In [17]:
# Our final model includes five features, which we expect to increase the score of our model on both 
# training and test data
preproc_final1 = ColumnTransformer(
        transformers = [
            ('year', OneHotEncoder(drop = "first"), ['published_year']),
            ('health', OneHotEncoder(drop = "first"), ['healthy'])],
        remainder='passthrough'
    )
pl_final1 = Pipeline([
        ("preprocessor", preproc_final1),
        ('decision-tree', DecisionTreeRegressor(max_depth = 5))
    ])

pl_final1.fit(X_train[["published_year", 'total fat (PDV)', 'n_ingredients', 'healthy'
                       , 'sugar (PDV)']], y_train)
pl_final1.score(X_train[["published_year", 'total fat (PDV)', 'n_ingredients', 'healthy'
                         , 'sugar (PDV)']], y_train)

0.8791757218512102

In [18]:
#run final model on test data
pl_final1.score(X_test[["published_year", 'total fat (PDV)', 'n_ingredients', 'healthy'
                        , 'sugar (PDV)']], y_test)

0.7385341055362334

We plan to tune the hyperparameter of the DecisionTreeRegressor, which is max_depth. We want to tune this using grids search because we want to find the best parameter that will optimize our model on the training data while having a similar score for unseen data. We don't want our model to be overly complicated.

In [19]:
# fine tuning hyperparameter for DecisionTreeRegressor to find the best max_depth.
hyperparameters = {
    'decision-tree__max_depth': np.arange(1, 30)
}
grids = GridSearchCV(pl_final1, hyperparameters, cv = 5)
grids.fit(X_train[["published_year", 'total fat (PDV)', 'n_ingredients', 'healthy'
                   , 'sugar (PDV)']], y_train)
grids.best_params_

{'decision-tree__max_depth': np.int64(9)}

In [20]:
# final version of our final model that includes the best parameter for max_depth.
preproc_final2 = ColumnTransformer(
        transformers = [
            ('year', OneHotEncoder(drop = "first"), ['published_year']),
            ('ingredients', Binarizer(), ['n_ingredients'])],
        remainder='passthrough'
    )
pl_final2 = Pipeline([
        ("preprocessor", preproc_final2),
        ('decision-tree', DecisionTreeRegressor(max_depth = 8))
    ])

pl_final2.fit(X_train[["published_year", 'total fat (PDV)', 'n_ingredients', 'healthy'
                       , 'sugar (PDV)']], y_train)
pl_final2.score(X_train[["published_year", 'total fat (PDV)', 'n_ingredients', 'healthy'
                         , 'sugar (PDV)']], y_train)

0.9226275117541688

In [21]:
#run tuned final model on test data
pl_final2.score(X_test[["published_year", 'total fat (PDV)', 'n_ingredients', 'healthy'
                        , 'sugar (PDV)']], y_test)

0.8214258278686812

### Fairness Analysis

> **Null Hypothesis**: The regressor's R squared value is the same for both old recipes and new recipes, and any differences are due to random chance.

> **Alternative Hypothesis**: The regressor's R squred value is higher for new recipes.

> **Test Statistic**: Difference in the R squared value for both group (old - new)

> **Significance level**: 0.05

In [22]:
# Here we defined a function to help us to separate recipes into two category, 
# old and new, based on the published year 
def year_sort(row):
    if (row["published_year"] >= 2008) & (row["published_year"] < 2012):
        return "old"
    if (row["published_year"] >= 2012) & (row["published_year"] <= 2018):
        return "new"
    return np.nan

cleaned_recipes["old_new"] = cleaned_recipes.apply (lambda row: year_sort(row), axis=1)

In [23]:
# calculate the observed test statistics. Our test statistics is the difference between 
# new and old recipes.
obs = cleaned_recipes.groupby('old_new').apply(lambda x: pl_final2.score(x[["published_year", 
                                                                            'total fat (PDV)', 
                                                                            'n_ingredients','healthy', 
                                                                            'sugar (PDV)']], 
                                                                         x['calories (#)'])
                                              ).diff().iloc[-1]
obs

/var/folders/rj/dpbwchcx3tz4lk0s_qynk5fw0000gn/T/ipykernel_12948/1634554436.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  obs = cleaned_recipes.groupby('old_new').apply(lambda x: pl_final2.score(x[["published_year",


np.float64(-0.008995926778622154)

In [24]:
# run permutation test to simulate the test
diff_in_acc = []
for _ in range(100):
    s = (
        cleaned_recipes[["published_year", 'total fat (PDV)', 'n_ingredients', 
                         'healthy', 'sugar (PDV)', 'calories (#)']]
        .assign(old_new_shuffle=cleaned_recipes.old_new.sample(frac=1.0, replace=False)
                .reset_index(drop=True))
        .groupby('old_new_shuffle')
        .apply(lambda x: pl_final2.score(x[["published_year", 'total fat (PDV)', 
                                            'n_ingredients', 'healthy', 'sugar (PDV)']], 
                                         x['calories (#)'])).diff().iloc[-1]
    )
    
    diff_in_acc.append(s)

p_value = (diff_in_acc <= obs).mean()

/var/folders/rj/dpbwchcx3tz4lk0s_qynk5fw0000gn/T/ipykernel_12948/1466094472.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pl_final2.score(x[["published_year", 'total fat (PDV)',
/var/folders/rj/dpbwchcx3tz4lk0s_qynk5fw0000gn/T/ipykernel_12948/1466094472.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pl_final2.score(x[["published_year", 'total fat (PDV)',
/var/f

In [25]:
# draw graph
fig = pd.Series(diff_in_acc).plot(kind='hist', histnorm='probability', nbins=20,
                            title='Difference in Score (Old - New)')
fig.add_vline(x=obs, line_color='red')
fig.update_layout(xaxis_range=[-0.1, 0.1])
fig.add_annotation(text='<span style="color:red">Observed Difference in Score</span>'
                   , x=-0.075,showarrow=False, y=0.17)

Since the p_value is greater than 0.05, we fail to reject the null. Thus we can answer that our model does not perform worse for old(before 2012) recipes than it does for new(after 2012) recipes based on the result of this one simulation.